In [17]:
!pip install scikit-video
!pip install opencv-python

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io

import cv2
import json
import random

from keras.engine.topology import Input
from keras.models import Sequential,model_from_json, Model
from keras.layers.core import Dense, Reshape, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning


__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The returns of the function **act** is an action that the agent needs to do based on probability $\epsilon$ and act according to  its policy with a probability $1-\epsilon$.

**epsilon** is essential for setting a proportion(the proportion of action that will be taken randomly) and epsilon stimulates the agent to explore. The algorithm would just return the policy learned by the agent with no  probability of this random action. By exploring other states through random actions, the agent can optimize the policy. In other words, to improve policies, we have to make sure the exploration to find state-action pairs is not under a deterministic policy. 

***

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state
#Dimension: 5 x 5 x 10

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=15 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array **position** shows the exact position of the rat(the agent) on the island(the board). Its position will be denoted by 1. The number -1 represent the boundaries of the board. The number 0 shows all other places that can be visited.


The array **board** shows the position of where the rewards are located. If one reward is eatten by the agent during an around, this reward is reset to 0. 
We will get differnet scores, according to the state(eatting a poison, nothing, eatting a cheese). 

## Random Agent

***

__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self,n_action=4):
        super(RandomAgent, self).__init__()
        
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    # Win - Lose    
    for e in range(1,epochs+1):
        
        ##### FILL IN HERE
        #At each epoch, a new game is started, and the state is started over.
        state = env.reset()
        game_over = False
        win, lose = 0, 0
        
        # when the time of ending(maximum game time) is coming, game over
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # an action, get to the next state, the reward
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
            
            # Apply the reinforcement strategy
            
        # Save as a mp4
        env.draw(prefix+str(e+1))
        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [148]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 11.0/7.0. Average score (2.0)
Win/lose count 5.5/6.0. Average score (1.1666666666666667)
Win/lose count 9.0/17.0. Average score (-1.125)
Win/lose count 10.0/10.0. Average score (-0.9)
Win/lose count 12.0/10.0. Average score (-0.4166666666666667)
Win/lose count 10.0/7.0. Average score (0.07142857142857142)
Win/lose count 7.0/16.0. Average score (-1.0625)
Win/lose count 10.5/21.0. Average score (-2.111111111111111)
Win/lose count 12.0/13.0. Average score (-2.0)
Win/lose count 13.5/12.0. Average score (-1.6818181818181819)
Win/lose count 10.5/17.0. Average score (-2.0833333333333335)
Win/lose count 11.0/19.0. Average score (-2.5384615384615383)
Win/lose count 10.0/14.0. Average score (-2.642857142857143)
Win/lose count 11.5/17.0. Average score (-2.8333333333333335)
Win/lose count 8.0/8.0. Average score (-2.65625)
Final score: -2.8333333333333335


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


**(1)**



$$Q^{\pi}(s, a) = E_{\pi} [\sum_{i=0}^{\infty}{\gamma^tr_t(s_t,\pi(s_t), s_{t+1})|s_t = s, a_t = a]}$$

According to the markovian property of each $(a_t, s_t)$ couple, we can also consider it as:


$$Q^\pi(s,a)=\gamma^{0}E_{p^{\pi}}[r(s, a|s_0=s, a_0=a)] + \gamma E_{p^{\pi}}[r(s_1, a_1)|a, s] + ... + \gamma^{T}E_{p^{\pi}}[r(s_T, a_T)|a_{T-1}, s_{T-1}]$$
where we have:

$$\gamma^{0}E_{p^{\pi}}[r(s, a|s_0=s, a_0=a)]=E_{p^{\pi}}[r(s, a|s, a)]=E_{p^{\pi}(.|s,a)}[r(s, a)]$$
also, by setting $s_1=s{'}$ and $a_1=a{'}$, we get by the definition of the $Q$ function:

$$\gamma E_{p^{\pi}}[r(s_1, a_1)|a, s] + ... + \gamma^{T}E_{p^{\pi}}[r(s_T, a_T)|a_{T-1}, s_{T-1}] = \gamma Q^{\pi}(s^{\prime}, a^{\prime})$$
Thus, as $\gamma Q^{\pi}(s^{\prime}, a^{\prime})$, is a sum of expectations:

$$E_{p(.|s,a)}[r(s, a)] + \gamma Q^{\pi}(s^{\prime}, a^{\prime}) = E_{(s',a')\sim p(.|s,a)}[r(s, a) + \gamma Q^{\pi}(s^{\prime}, a^{\prime})]$$







**(2)**

Under the optimal policy, let's see $S$ as the set of all possible states, we have :

$$\underset{s' \epsilon S}{max} ({p^{\pi^*}(.|s, a)) = s'}$$ 

thus:

$$s'\sim \pi^*(.|s, a)$$

Also, for denoting the Q function under the optimal policy, we rewrite simply:
$$Q^\pi(s', a')=Q^*(s', a')$$

Considering now s' as given with $(s_0 = s, a_0 = a)$ for $\pi^*$, along with $r(s, a)$, we need to maximize  $Q^*$ $w.r.t.$ $a'$.

That is $$max(E_{(s', a') \sim p^{\pi^*}(.|s, a)}[r(s, a)+\gamma Q^*(s', a')]) \Leftrightarrow E_{s' \sim \pi^*(s, a)}[r(s, a)+\gamma  \underset{a'}{max}(Q^*(s', a'))]$$


**(3)**

Given the loss function $\mathcal{L}(\theta) = [Q^*(s, a, \theta) - Q(s, a, \theta)]^2$

By substituting $Q^*(s, a, \theta)$, we got:

$$\Rightarrow \mathcal{L}(\theta) = E_{s'\sim {\pi^*}(.|s, a)|}|| r + \gamma \underset{a'}{max maxQ}(s', a', \theta)-Q(s, a, \theta)||^2 $$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        pass

    def random_access(self):
        r= np.random.randint(0, len(self.memory))
        return self.memory[r]


***
The pipeline we will use for training is given below:

In [0]:

def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        acts_ = self.model.predict(np.reshape(s,(1,5,5,self.n_state)))
        return np.argmax(acts_[0])
        ######
        #return np.argmax(self.model.predict(s.reshape(1, 5, 5, 2)))
        

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            current_state, next_state, action, reward, game_over_ = self.memory.random_access() #sample some moves
            input_states[i] = current_state
            
            s_reshape=np.reshape(current_state,(1,5,5,self.n_state))
            next_state=np.reshape(next_state,(1,5,5,self.n_state))
            target_q[i] = self.model.predict(s_reshape)
            
           
            if game_over_:

                target_q[i,a_]= reward
            else:
                q = self.model.predict(next_state)
                target_q[i,a_] = reward + self.discount * np.max(q)
        
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        
        # NN Model
        model = Sequential()
        # Flattening the input frame
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(24,init='uniform', activation='relu'))
        model.add(Dense(4,init='uniform', activation='relu'))
        

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
        
        
        
      

In [0]:
size = 13
T=200
temperature=0.3
epochs_train=30
epochs_test=15

In [153]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.2, memory_size=16000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train_')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(24, activation="relu", kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="uniform")`


Epoch 000/030 | Loss 0.0226 | Win/lose count 3.5/6.0 (-2.5)
Epoch 001/030 | Loss 0.0263 | Win/lose count 4.0/8.0 (-4.0)
Epoch 002/030 | Loss 0.0125 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/030 | Loss 0.0100 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/030 | Loss 0.0020 | Win/lose count 2.0/4.0 (-2.0)
Epoch 005/030 | Loss 0.0171 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/030 | Loss 0.0226 | Win/lose count 6.5/6.0 (0.5)
Epoch 007/030 | Loss 0.0098 | Win/lose count 4.0/10.0 (-6.0)
Epoch 008/030 | Loss 0.0001 | Win/lose count 5.5/6.0 (-0.5)
Epoch 009/030 | Loss 0.0077 | Win/lose count 2.5/10.0 (-7.5)
Epoch 010/030 | Loss 0.0001 | Win/lose count 5.0/8.0 (-3.0)
Epoch 011/030 | Loss 0.0155 | Win/lose count 3.5/8.0 (-4.5)
Epoch 012/030 | Loss 0.0094 | Win/lose count 10.0/7.0 (3.0)
Epoch 013/030 | Loss 0.0114 | Win/lose count 3.5/9.0 (-5.5)
Epoch 014/030 | Loss 0.0173 | Win/lose count 6.0/12.0 (-6.0)
Epoch 015/030 | Loss 0.0310 | Win/lose count 7.0/10.0 (-3.0)
Epoch 016/030 | Loss 0.0019 | Win/lose 

In [154]:
HTML(display_videos('fc_train_30.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
     
        model = Sequential()
        model.add(Conv2D(32,(2,2),activation='relu',input_shape=(5,5,self.n_state)))
        #model.add(Conv2D(16,(1,1),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(32,(2,2), activation='relu'))
        #model.add(Conv2D(32,(1,1), activation='relu'))
        model.add(Flatten())
        
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        print(model.summary())
        self.model = model

In [156]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train_')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_45 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 4)                 1156      
Total params: 5,572
Trainable params: 5,572
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/030 | Loss 0.0068 | Win/lose count 2.5/7.0 (-4.5)
Epoch 001/030 | Loss 0.0043 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/030 | Loss 0.0107 | Win/lose count 7.0/8.0 (-1.0)
Epoch 003/030 | Loss 0.0080 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/030 | Loss 0.0185 

In [157]:
HTML(display_videos('cnn_train_30.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

**Testing  with temperature set to 0.3**

In [158]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_model.h5',name_model='cnn_train_model.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_train_model.h5',name_model='fc_train_model.json')
print('\n ___________Test of the CNN_____________')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\n _____________Test of the FC___________________')
test(agent_fc,env,epochs_test,prefix='fc_test')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_46 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 4)                 1156      
Total params: 5,572
Trainable params: 5,572
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(24, activation="relu", kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="uniform")`



 ___________Test of the CNN_____________
Win/lose count 3.0/8.0. Average score (-2.5)
Win/lose count 2.5/1.0. Average score (-1.1666666666666667)
Win/lose count 2.0/4.0. Average score (-1.375)
Win/lose count 2.0/1.0. Average score (-0.9)
Win/lose count 3.5/3.0. Average score (-0.6666666666666666)
Win/lose count 2.5/7.0. Average score (-1.2142857142857142)
Win/lose count 3.0/6.0. Average score (-1.4375)
Win/lose count 5.5/2.0. Average score (-0.8888888888888888)
Win/lose count 2.0/3.0. Average score (-0.9)
Win/lose count 3.0/1.0. Average score (-0.6363636363636364)
Win/lose count 4.0/4.0. Average score (-0.5833333333333334)
Win/lose count 2.5/1.0. Average score (-0.4230769230769231)
Win/lose count 0.5/2.0. Average score (-0.5)
Win/lose count 5.0/3.0. Average score (-0.3333333333333333)
Win/lose count 5.0/4.0. Average score (-0.25)
Final score: -0.26666666666666666

 _____________Test of the FC___________________
Win/lose count 2.0/7.0. Average score (-2.5)
Win/lose count 3.0/8.0. Avera

**Testing with temperature set to 0.9**

In [159]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_model.h5',name_model='cnn_train_model.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_train_model.h5',name_model='fc_train_model.json')
print('\n ___________Test of the CNN_____________')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\n _____________Test of the FC___________________')
test(agent_fc,env,epochs_test,prefix='fc_test')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_48 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_85 (Dense)             (None, 4)                 1156      
Total params: 5,572
Trainable params: 5,572
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(24, activation="relu", kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="uniform")`



 ___________Test of the CNN_____________
Win/lose count 7.0/2.0. Average score (2.5)
Win/lose count 5.0/1.0. Average score (3.0)
Win/lose count 8.0/0. Average score (4.25)
Win/lose count 6.5/1.0. Average score (4.5)
Win/lose count 7.0/1.0. Average score (4.75)
Win/lose count 6.0/4.0. Average score (4.357142857142857)
Win/lose count 9.0/2.0. Average score (4.6875)
Win/lose count 4.0/4.0. Average score (4.166666666666667)
Win/lose count 9.5/2.0. Average score (4.5)
Win/lose count 3.0/2.0. Average score (4.181818181818182)
Win/lose count 12.5/2.0. Average score (4.708333333333333)
Win/lose count 9.5/1.0. Average score (5.0)
Win/lose count 6.0/0. Average score (5.071428571428571)
Win/lose count 7.0/0. Average score (5.2)
Win/lose count 11.0/1.0. Average score (5.5)
Final score: 5.866666666666666

 _____________Test of the FC___________________
Win/lose count 9.0/3.0. Average score (3.0)
Win/lose count 9.0/2.0. Average score (4.333333333333333)
Win/lose count 11.0/1.0. Average score (5.75)

Firstly,  one thing interesting is that the exploration is not well executed, sometime the exploration process is stuck for no positive reward in the agent's vision. In addition,  the results of CNN are more stable than the one of FC. 

Secondly, we can notice that under different tempetatures(0.3, 0.9), the performances are quite different.

With 0.3 temperature, the scores of CNN and FC are much lower than the ones with 0.9 temperature. It seems that the higher teemperature, the better the final score, but we should consider other possible reasons. The one reason of this situation is that the possibility of getting cheese is increased and the possibility of poisonous cells is lowered.  



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
       
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
      
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1
#
        self.malus_position = np.zeros((grid_size,grid_size))      
#         
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
#       
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1 
        reward = reward + self.board[self.x, self.y]
#        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), #OK ajouté dans init
                                self.board.reshape(self.grid_size, self.grid_size,1), #OK deja dans init
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)#OK deja init
#        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), #OK ajouté dans init
                                        self.board.reshape(self.grid_size, self.grid_size,1), #OK deja dans init
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)#OK deja init
#
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
  
    

In [0]:
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=35
epochs_test=15

In [162]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore35.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_50 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_88 (Dense)             (None, 4)                 1156      
Total params: 5,700
Trainable params: 5,700
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/035 | Loss 0.0271 | Win/lose count 7.5/24.70000000000008 (-17.20000000000008)
Epoch 001/035 | Loss 0.0019 | Win/lose count 2.6/20.20000000000002 (-17.60000000000002)
Epoch 002/035 | Loss 0.0147 | Win/lose count 3.8/23.300000000000058 (-19.500000000000057)
E

In [163]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore_')
HTML(display_videos('cnn_test_explore_15.mp4'))

Win/lose count 4.3999999999999995/24.00000000000005. Average score (-9.800000000000026)
Win/lose count 2.4/21.500000000000043. Average score (-12.900000000000032)
Win/lose count 3.9999999999999996/23.100000000000062. Average score (-14.450000000000038)
Win/lose count 5.6000000000000005/25.700000000000077. Average score (-15.580000000000046)
Win/lose count 2.0/21.600000000000023. Average score (-16.250000000000043)
Win/lose count 1.2000000000000002/20.800000000000026. Average score (-16.728571428571467)
Win/lose count 4.8/26.900000000000116. Average score (-17.40000000000005)
Win/lose count 0.4/23.000000000000057. Average score (-17.977777777777828)
Win/lose count 2.4/22.500000000000053. Average score (-18.190000000000047)
Win/lose count 4.3999999999999995/23.000000000000068. Average score (-18.227272727272776)
Win/lose count 2.8/22.40000000000005. Average score (-18.341666666666715)
Win/lose count 2.8/22.40000000000006. Average score (-18.43846153846159)
Win/lose count 3.19999999999999